In [1]:
import numpy as np
import tensornetwork as tn

In [47]:
# CX gate
CX = tn.Node(np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
]).reshape(2, 2, 2, 2))

In [95]:
# Random 4x4 unitary matrix
U = tn.Node(np.random.rand(4, 4).reshape(2, 2, 2, 2))
U.tensor.shape

(2, 2, 2, 2)

In [49]:
def untangle(U):  # 2, 2, 2, 2
    U = U.copy()
    mtr = U.tensor[:, 0, :, :].reshape(2, 4)
    v = mtr[0] - mtr[1]
    v = 1 / np.array([v[0] + v[3], v[2] - v[1]])
    v = v / np.linalg.norm(v)
    v = np.array([v[0], v[1], -v[1], v[0]]).repeat(2)
    Z = tn.Node(np.array([1, 0]))
    kd = tn.Node(np.array([1, 0, 0, 0, 0, 0, 0, 1]).reshape(2, 2, 2))
    T = tn.Node(v.reshape(2, 2, 2))
    tn.connect(U[1], Z[0])
    tn.connect(U[2], kd[0])
    tn.connect(U[3], T[1])
    tn.connect(T[0], kd[1])
    R = tn.contractors.auto([U, Z, kd, T], output_edge_order=[U[0], kd[2], T[2]])
    return R


In [79]:
V = untangle(U)
Vf = V.tensor.reshape(4, 2)
print("rank", np.linalg.matrix_rank(Vf))

print(np.pow(V.tensor.reshape(2, 4), 2).sum(axis=0))

rank 1
[0.19624682 0.19624682 0.45787439 0.45787439]


In [83]:
print(np.pow(U.tensor[:, 0].reshape(4, 2), 2).sum(axis=1)) 

[0.600493   0.1133945  0.73987921 0.70893583]


In [99]:
T = U.tensor.copy()
T = T[:, 0]
T = T ** 2
T = T.sum(axis=2)
T.flatten()


array([0.02711404, 0.50097418, 0.24863283, 0.23912974])